In [1]:
import pandas as pd
import numpy as np

PATH = "data/74486094789.csv"

df = pd.read_csv(
    PATH,
    parse_dates=["DATE"],
)

df.head(), df.shape


C:\Users\User\AppData\Local\Temp\ipykernel_18032\3849994107.py:6: DtypeWarning: Columns (0: AL1, 1: MW2, 2: MW3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


(       STATION                DATE  SOURCE  LATITUDE  LONGITUDE  ELEVATION  \
 0  74486094789 2023-01-01 00:00:00       4  40.63915   -73.7639        2.7   
 1  74486094789 2023-01-01 00:51:00       7  40.63915   -73.7639        2.7   
 2  74486094789 2023-01-01 01:49:00       6  40.63915   -73.7639        2.7   
 3  74486094789 2023-01-01 01:51:00       7  40.63915   -73.7639        2.7   
 4  74486094789 2023-01-01 02:03:00       7  40.63915   -73.7639        2.7   
 
                                NAME REPORT_TYPE CALL_SIGN QUALITY_CONTROL  \
 0  JFK INTERNATIONAL AIRPORT, NY US       FM-12     99999            V020   
 1  JFK INTERNATIONAL AIRPORT, NY US       FM-15     KJFK             V030   
 2  JFK INTERNATIONAL AIRPORT, NY US       FM-16     KJFK             V030   
 3  JFK INTERNATIONAL AIRPORT, NY US       FM-15     KJFK             V030   
 4  JFK INTERNATIONAL AIRPORT, NY US       FM-16     KJFK             V030   
 
    ...  OC1  OD1  OE1  OE2  OE3  RH1  RH2  RH3  \
 0 

In [3]:
df["hour"] = df["DATE"].dt.floor("h")

df[["DATE", "hour"]].head()


,DATE,hour
0,2023-01-01 00:00:00,2023-01-01 00:00:00
1,2023-01-01 00:51:00,2023-01-01 00:00:00
2,2023-01-01 01:49:00,2023-01-01 01:00:00
3,2023-01-01 01:51:00,2023-01-01 01:00:00
4,2023-01-01 02:03:00,2023-01-01 02:00:00


In [5]:
# TMP: air temperature, tenths of °C, +9999 = missing
tmp = df["TMP"].str.split(",", expand=True)
df["air_temp_raw"] = pd.to_numeric(tmp[0], errors="coerce")
df.loc[df["air_temp_raw"] == 9999, "air_temp_raw"] = pd.NA
df["air_temp_c"] = df["air_temp_raw"] / 10

# SLP: sea level pressure, tenths of hPa, 99999 = missing
slp = df["SLP"].str.split(",", expand=True)
df["slp_raw"] = pd.to_numeric(slp[0], errors="coerce")
df.loc[df["slp_raw"] == 99999, "slp_raw"] = pd.NA
df["slp_hpa"] = df["slp_raw"] / 10

df[["TMP", "air_temp_c", "SLP", "slp_hpa"]].head()


,TMP,air_temp_c,SLP,slp_hpa
0,"+0100,1",10.0,"10108,1",1010.8
1,"+0089,5",8.9,"10102,5",1010.2
2,"+0090,5",9.0,"99999,9",NaN
3,"+0094,5",9.4,"10092,5",1009.2
4,"+0094,5",9.4,"99999,9",NaN


In [6]:
audit = {
    "rows": len(df),
    "unique_hours": df["hour"].nunique(),
    "temp_missing_%": round(df["air_temp_c"].isna().mean() * 100, 2),
    "slp_missing_%": round(df["slp_hpa"].isna().mean() * 100, 2),
    "temp_min": df["air_temp_c"].min(),
    "temp_max": df["air_temp_c"].max(),
}

audit


{'rows': 13423,
 'unique_hours': 8760,
 'temp_missing_%': np.float64(2.85),
 'slp_missing_%': np.float64(13.6),
 'temp_min': np.float64(-15.0),
 'temp_max': np.float64(33.9)}